In [1]:
#basic imports
import glob
import IPython
from random import randint
import pickle
#data processing
import os, librosa
import numpy as np
from tqdm import tqdm
import scipy.io.wavfile
import python_speech_features as psf
from sklearn.preprocessing import scale
#modelling
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

In [3]:
duration=3
sr=16000

In [4]:
IPython.display.Audio(r"dataset/1/00001.wav", rate=sr)

ValueError: could not convert string to float: 'dataset/1/00001.wav'

In [ ]:
parent_dir = 'dataset/'
data = []
labels = []
for sub_dirs in tqdm(os.listdir(parent_dir)):
    sub_dir = parent_dir + sub_dirs
    for files in os.listdir(sub_dir):
        file_path = sub_dir + '/' + files
        input_length=sr*duration
        y, sr = librosa.load(file_path,sr=sr, duration=duration)
        dur = librosa.get_duration(y=y)
        if (round(dur) < duration):
            y = librosa.util.fix_length(y, round(input_length)) 
        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        label = file_path.split('/')[-2]
        data.append(ps)
        labels.append(label)

In [ ]:
data = np.array(data)#, dtype='float32') / 255.0
labels = np.array(labels,dtype='int32' )
print(data.shape,labels.shape)

In [ ]:
pickle.dump(data, open("data.pkl","wb"))
pickle.dump(labels, open("labels.pkl","wb"))

In [5]:
data = pickle.load(open("data.pkl","rb"))
labels = pickle.load(open("labels.pkl","rb"))

In [6]:
def make_pairs(data, labels):
    pairData = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    for idxA in range(len(data)):
        currentAudio = data[idxA]
        label = labels[idxA]
        idxB = np.random.choice(idx[label])
        posData = data[idxB]
        pairData.append([currentAudio, posData])
        pairLabels.append([1])
        negIdx = np.where(labels != label)[0]
        negData = data[np.random.choice(negIdx)]
        pairData.append([currentAudio, negData])
        pairLabels.append([0])
    return (np.array(pairData), np.array(pairLabels))

In [7]:
%%time
# prepare the positive and negative pairs
print("[INFO] preparing positive and negative pairs...")
(pairdata, pairlabels) = make_pairs(data, labels)

[INFO] preparing positive and negative pairs...
CPU times: user 3.69 s, sys: 1.1 s, total: 4.79 s
Wall time: 4.8 s


In [8]:
trainx, testx, trainy, testy = train_test_split(pairdata, pairlabels, test_size=0.25, random_state=42)
print(trainx.shape, testx.shape, trainy.shape, testy.shape)

(51672, 2, 128, 94) (17224, 2, 128, 94) (51672, 1) (17224, 1)


In [9]:
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation,Subtract , MaxPooling2D
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM# , CuDNNLSTM, GRU, CuDNNGRU
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

2023-06-09 18:46:00.231609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 18:46:01.053880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9885 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:02:00.0, compute capability: 8.6


In [10]:
if tf.test.is_gpu_available():
    print('GPU available: True')
else:
    print('GPU available: False')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: True


2023-06-09 18:46:01.071840: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 9885 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:02:00.0, compute capability: 8.6


In [11]:
def build_base_network(input_shape):
    input = Input(shape=input_shape)
    x = LSTM(256,return_sequences=False)(input)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

In [12]:
input_dim = trainx.shape[2:]
print(input_dim)
audio_a = Input(shape=input_dim)
audio_b = Input(shape=input_dim)

(128, 94)


In [13]:
base_network = build_base_network(input_dim)

feat_vecs_a = base_network(audio_a)
feat_vecs_b = base_network(audio_b)

2023-06-09 18:46:01.120139: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9885 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:02:00.0, compute capability: 8.6


In [14]:
# Define the Euclidean distance function
def euclidean_distance(inputs):
    x, y = inputs
    sum_squared = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(sum_squared)

In [15]:
from sklearn.metrics.pairwise import euclidean_distances

from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization

distance = Lambda(euclidean_distance)([feat_vecs_a, feat_vecs_b])
distance = Lambda(lambda x: K.abs(x))(distance)
merged = Dropout(0.2)(distance)
merged = Dense(64, activation='relu')(merged)

In [16]:
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[audio_a, audio_b], outputs=outputs)

In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 94)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128, 94)]    0           []                               
                                                                                                  
 model (Functional)             (None, 128)          392320      ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

In [18]:
model.compile(loss="mse", optimizer="rmsprop",metrics=["mse"])
# train the model
print("[INFO] training model...")
history = model.fit([trainx[:, 0], trainx[:, 1]], trainy[:],
                    validation_data=([testx[:, 0], testx[:, 1]], testy[:]),
                    batch_size=64, 
                    epochs=100)

[INFO] training model...
Epoch 1/100


2023-06-09 18:46:15.728149: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200


  1/808 [..............................] - ETA: 1:09:11 - loss: 0.2533 - mse: 0.2533

2023-06-09 18:46:16.734916: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


808/808 [==============================] - 28s 28ms/step - loss: 0.2501 - mse: 0.2501 - val_loss: 0.2500 - val_mse: 0.2500
Epoch 2/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2418 - mse: 0.2418 - val_loss: 0.2383 - val_mse: 0.2383
Epoch 3/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2332 - mse: 0.2332 - val_loss: 0.2413 - val_mse: 0.2413
Epoch 4/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2311 - mse: 0.2311 - val_loss: 0.2236 - val_mse: 0.2236
Epoch 5/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2254 - mse: 0.2254 - val_loss: 0.2238 - val_mse: 0.2238
Epoch 6/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2148 - mse: 0.2148 - val_loss: 0.2136 - val_mse: 0.2136
Epoch 7/100
808/808 [==============================] - 19s 24ms/step - loss: 0.2044 - mse: 0.2044 - val_loss: 0.2043 - val_mse: 0.2043
Epoch 8/100
808/808 [==============================] - 19s 24ms/ste

808/808 [==============================] - 19s 24ms/step - loss: 0.0305 - mse: 0.0305 - val_loss: 0.2017 - val_mse: 0.2017
Epoch 62/100
808/808 [==============================] - 19s 24ms/step - loss: 0.0301 - mse: 0.0301 - val_loss: 0.2000 - val_mse: 0.2000
Epoch 63/100
808/808 [==============================] - 19s 24ms/step - loss: 0.0282 - mse: 0.0282 - val_loss: 0.2039 - val_mse: 0.2039
Epoch 64/100
808/808 [==============================] - 19s 24ms/step - loss: 0.0280 - mse: 0.0280 - val_loss: 0.2043 - val_mse: 0.2043
Epoch 65/100
808/808 [==============================] - 19s 24ms/step - loss: 0.0275 - mse: 0.0275 - val_loss: 0.2035 - val_mse: 0.2035
Epoch 66/100
808/808 [==============================] - 19s 24ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.2055 - val_mse: 0.2055
Epoch 67/100
808/808 [==============================] - 20s 24ms/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.2030 - val_mse: 0.2030
Epoch 68/100
808/808 [==============================] - 20s 2

In [19]:
model.evaluate([testx[:, 0], testx[:, 1]],testy[:])

539/539 [==============================] - 5s 10ms/step - loss: 0.2089 - mse: 0.2089


[0.2088661640882492, 0.2088661640882492]

In [20]:
model.save("voice_siamese.h5")

In [21]:
def audio2vector(file_path, duration=3, sr=16000):
    input_length=sr*duration
    y, sr = librosa.load(file_path,sr=sr, duration=duration)
    dur = librosa.get_duration(y=y)
    if (round(dur) < duration):
        y = librosa.util.fix_length(y, round(input_length))      
    ps = librosa.feature.melspectrogram(y=y, sr=sr)
    return ps

In [22]:
test = pickle.load(open("test_data.pkl", 'rb'))

In [23]:
voice1 = np.expand_dims(test[5426],axis=0)
voice2 = np.expand_dims(test[5426],axis=0)

pred = model.predict([voice1,voice2])[0]
print(pred)

1/1 [==============================] - 1s 796ms/step
[0.9999968]


In [24]:
voice1 = np.expand_dims(test[5426],axis=0)
voice2 = np.expand_dims(test[300],axis=0)

pred = model.predict([voice1,voice2])[0]
print(pred)

1/1 [==============================] - 0s 25ms/step
[0.99297523]


In [26]:
values = []
voice1 = np.expand_dims(test[5426],axis=0)
for audio in tqdm(test):
    voice2 = np.expand_dims(audio,axis=0)
    values.append(model.predict([voice1,voice2])[0][0])

  0%|                                                  | 0/5435 [00:00<?, ?it/s]

1/1 [==============================] - 0s 30ms/step


  0%|                                          | 2/5435 [00:00<07:50, 11.54it/s]

1/1 [==============================] - 0s 23ms/step


  0%|                                          | 4/5435 [00:00<18:22,  4.93it/s]

...

100%|██████████████████████████████████████▉| 5432/5435 [06:45<00:00, 13.56it/s]

1/1 [==============================] - 0s 31ms/step


100%|██████████████████████████████████████▉| 5434/5435 [06:45<00:00, 13.70it/s]

1/1 [==============================] - 0s 30ms/step


100%|███████████████████████████████████████| 5435/5435 [06:45<00:00, 13.40it/s]


In [27]:
np.argmax(values)

5426